In [ ]:
import pandas as pd
import re
import csv

### Outlier decetion of internal data

In [ ]:
pdObj = pd.read_json('../data/curated/domain.json', orient='index')
pdObj.to_csv('streaming_test.csv', index=True)

In [ ]:
# Converte json document to csv.
pdObj = pd.read_json('domain.json', orient='index')
pdObj.to_csv('streaming_test.csv', index=True)

In [ ]:
# Get the cost
df = pd.read_csv('streaming_test.csv')
df['cost'] = df['cost_text']
for i in range(len(df)):
    df['cost'].iloc[i] = re.findall(r'\d+\.?\d*',df['cost_text'].iloc[i])

In [ ]:
# Delete missing values
delete_list = []
df_new = df.copy()
for i in range(len(df)):
    if len(df['cost'].iloc[i]) == 0:
        delete_list.append(i)
df_new = df_new.drop(df_new.index[delete_list])

In [ ]:
# Transform the data type of cost into float.
for i in range(len(df_new)):
    df_new['cost'].iloc[i] = float(df_new['cost'].iloc[i][0])
    
df_new['cost'] = df_new['cost'].astype('float')

In [ ]:
# Delete the extreme values
delete_list = []
k1 = 0.8
k2 = 0.2
df_new2 = df_new.copy()
for i in range(len(df_new2)):
    if df_new2['cost'].iloc[i] < df_new2['cost'].mean()*(1-k1) or df_new2['cost'].iloc[i] > df_new2['cost'].mean()*(1+k2):
        delete_list.append(i)
df_new2 = df_new2.drop(df_new2.index[delete_list])
df_new2.describe()

In [ ]:
df = pd.read_csv("../data/raw/2021_GCP_SA2_for_VIC_short-header/2021 Census GCP Statistical Area 2 for VIC/2021Census_G33_VIC_SA2.csv")
df.head()

In [ ]:
cols = [col for col in df.columns if 'Tot' in col and 'Neg' not in col and 'stated' not in col]

In [ ]:
df1 = df[cols].drop(['Tot_Family_households','Tot_Non_family_households','Tot_Tot'],axis=1)
df1['SA2_CODE21'] = df['SA2_CODE_2021']
df1.head()

In [ ]:
df1.columns[0]

In [ ]:
avg = 0
df1['total_amount'] = 0
df1['family_amount'] = 0
for i in range(len(df1.columns)-3):
    if(i != 15):
        low = int(re.findall(r'[0-9]+', df1.columns[i])[0])
        upper = int(re.findall(r'[0-9]+', df1.columns[i])[1])
        avg = low + ((upper - low)/2)
    else:
        avg = 5500
    df1['total_amount'] = df1['total_amount'] + avg * df1.iloc[:, i]
    df1['family_amount'] = df1['family_amount'] + df1.iloc[:, i]
df1['avg'] = df1['total_amount']/df1['family_amount']

In [ ]:
df1.head()

In [ ]:
df2 = df1[['SA2_CODE21','avg']]
df2

In [ ]:
df2.to_csv('../data/curated/income.csv',index=False)

### Locality to SA2 Coding Index

In [ ]:
index = pd.read_csv("../data/raw/australian_postcodes.csv")
sa2_2021 = pd.read_csv("../data/raw/CG_SA2_2016_SA2_2021.csv")

In [ ]:
len(set(sa2_2021['SA2_MAINCODE_2016']))

In [ ]:
index_2021 = sa2_2021.merge(index, on='SA2_MAINCODE_2016')
index_2021 = index_2021[index_2021['state'] == 'VIC']

In [ ]:
index2 = index_2021[['SA2_MAINCODE_2016', 'SA2_NAME_2016_x', 'SA2_CODE_2021', 'SA2_NAME_2021','postcode','locality','Lat_precise','Long_precise']]
index2 = index2.drop_duplicates(subset=['postcode'])

In [ ]:
len(set(index2['postcode']))

In [ ]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv')
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic[postcode['accuracy'] == 4]
code_vic = code_vic.drop_duplicates(subset=['postcode'])

In [ ]:
index_vic = code_vic[['postcode','latitude','longitude']].merge(index2, on='postcode')

In [ ]:
index_vic
index_vic.to_csv('../data/curated/postcode_to_sa2.csv',index=False)

In [ ]:
index_vic.count()

In [ ]:
domain = pd.read_csv("../data/curated/domain.csv")

In [ ]:
domain.loc[domain['postcode'] == 3337, 'suburb_sold'] = 99
domain.loc[domain['postcode'] == 3337, 'suburb_avg_day'] = 43
domain.loc[domain['postcode'] == 3337, 'suburb_population'] = 17589
domain.loc[domain['postcode'] == 3337, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3337, 'suburb_ower'] = '74%'
domain.loc[domain['postcode'] == 3337, 'suburb_renter'] = '26%'
domain.loc[domain['postcode'] == 3337, 'suburb_family'] = '46%'
domain.loc[domain['postcode'] == 3337, 'suburb_single'] = '54%'


In [ ]:
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_sold'].isnull()), 'suburb_sold'] = 97
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_avg_day'].isnull()),'suburb_avg_day'] = 14
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_population'].isnull()),'suburb_population'] = 8343
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_age'].isnull()),'suburb_age'] = '60+'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_ower'].isnull()), 'suburb_ower'] = '80%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_renter'].isnull()),'suburb_renter'] = '20%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_family'].isnull()),'suburb_family'] = '51%'
domain.loc[(domain['postcode'] == 3043)&(domain['suburb_single'].isnull()),'suburb_single'] = '49%'


In [ ]:
domain[domain['postcode'] == 3350].count()

In [ ]:
domain.loc[(domain['postcode'] == 3000)&(domain['suburb_population'].isnull())]

In [ ]:
post_n = set(domain[domain['suburb_population'].isnull()]['postcode'])

In [ ]:
for post in post_n:
    if post not in [3004,3195]:
        domain =  domain.drop(domain[(domain['postcode'] == post)&(domain['suburb_population'].isnull())].index)

In [ ]:
domain[domain['suburb_population'].isnull()]['postcode'].value_counts()

In [ ]:
domain.to_csv('../data/curated/domain_2.csv',index = False)

In [ ]:
domain2 = pd.read_csv("../data/curated/domain_4.csv")

In [ ]:
domain3 = pd.read_csv("../data/curated/df_property_result.csv")